In [1]:
# Cell 1: Install & Auth (Google AI Studio Version)
# ==============================================================================
print("📦 Installing dependencies...")
# We switch from 'vertexai' to the simpler 'google-generativeai' library
%pip install -q google-generativeai pydantic

import os
from google.colab import userdata
import google.generativeai as genai

try:
    print("🔑 Authenticating with API Key...")
    # 1. Fetch Key from Secrets
    api_key = userdata.get('GOOGLE_API_KEY')

    # 2. Configure the library
    genai.configure(api_key=api_key)

    print("✅ Success! Authenticated with Google AI Studio.")

except Exception as e:
    print(f"❌ Auth Failed: {e}")
    print("Did you create a secret named 'GOOGLE_API_KEY' in the 🔑 tab?")

📦 Installing dependencies...
🔑 Authenticating with API Key...
✅ Success! Authenticated with Google AI Studio.


In [2]:
!ls

adhd-assistant-capstone  sample_data


In [ ]:
import os

# 1. Navigate into the project folder
if os.path.exists("adhd-assistant-capstone"):
    %cd adhd-assistant-capstone
    print("📂 Navigated to project folder.")
else:
    print("⚠️ Folder not found. Cloning now...")
    !git clone https://github.com/viveksahukar/adhd-assistant-capstone.git
    %cd adhd-assistant-capstone

# 2. Re-create the missing setup_config.py file
# (We do this to ensure it exists even if it wasn't pushed to GitHub)
with open("setup_config.py", "w") as f:
    f.write('''
import os
from google.colab import userdata
from google.oauth2 import service_account
from google.cloud import aiplatform as vertexai
import json

def initialize_environment(project_id: str):
    print("--- 🚀 Starting Cloud-Native Authentication ---")

    try:
        # 1. Get the JSON string from Colab Secrets
        # Make sure the name inside get() matches your Secret name exactly!
        key_json = userdata.get('GCP_CREDENTIALS')

        # 2. Convert that string into a format Google's auth library understands
        key_info = json.loads(key_json)
        credentials = service_account.Credentials.from_service_account_info(key_info)

        # 3. Initialize Vertex AI
        os.environ["GCP_PROJECT_ID"] = project_id
        vertexai.init(
            project=project_id,
            location="us-central1",
            credentials=credentials
        )

        print("✅ Success! Authenticated using Colab Secrets.")
        print(f"Service Account: {credentials.service_account_email}")

    except Exception as e:
        print(f"❌ Auth Failed: {e}")
        print("Check: Did you create a secret named 'GCP_CREDENTIALS' in the 🔑 tab?")

    return None, os, vertexai
''')
print("✅ setup_config.py created/verified.")

# 3. Install Dependencies
print("📦 Installing dependencies...")
%pip install -q -r requirements.txt

# 4. Run your Initialization
print("🚀 Initializing Project...")
from setup_config import initialize_environment

# Replace with your project ID
PROJECT_ID = "adhd-assistant-capstone"
auth, os, vertexai = initialize_environment(PROJECT_ID)

/content/adhd-assistant-capstone
📂 Navigated to project folder.
✅ setup_config.py created/verified.
📦 Installing dependencies...


In [ ]:
# Cell 2: Project Initialization
# =================================================================
from setup_config import initialize_environment

# Your specific Project ID
PROJECT_ID = "adhd-assistant-capstone"

# Initialize Vertex AI and auth
# This returns the modules if you need to inspect them, but mainly sets up the global state.
auth, os, vertexai = initialize_environment(PROJECT_ID)

In [ ]:
# Cell 3: Import Agent Modules
# =================================================================
try:
    from agents import ConversationManagerAgent, TaskLogicAgent, ToolExecutionAgent
    print("✅ Successfully imported Agent classes.")
except ImportError as e:
    print(f"❌ Error importing agents: {e}")
    print("Make sure agents.py and tools.py are in the current folder.")

In [ ]:
# Cell 4: Create Agent Instances
# =================================================================
# 1. Create the Specialist Agents
# Note: TaskLogicAgent now defaults to 'gemini-2.5-pro' per our refinement
task_agent = TaskLogicAgent()
tool_agent = ToolExecutionAgent()

# 2. Create the Supervisor (Conversation Manager)
# This injects the specialists into the coordinator
manager = ConversationManagerAgent(task_agent=task_agent, tool_agent=tool_agent)

print("✅ Agent System Online. Ready for user input.")

In [ ]:
# Cell 5: Simulate User Interaction
# =================================================================

# 1. Define the User's "Brain Dump"
user_input = (
    "I have a big presentation on Friday morning at 10 AM. "
    "I need to call the doctor sometime next week to make an appointment, "
    "and I also have to remember to buy eggs and bread today."
)

print(f"👤 USER SAYS:\n{user_input}\n")
print("-" * 60)

# 2. Handle the message (with auto_confirm=False to test HITL)
# This mimics the "Think" step where the agent plans but waits for approval
agent_turn = manager.handle_user_message(
    user_text=user_input,
    user_id="notebook_test_user_01",
    auto_confirm=False
)

# 3. Display the Agent's proposed response
print(f"🤖 AGENT RESPONDS:\n{agent_turn.user_facing_message}")

In [ ]:
# Cell 6: User Confirmation & Tool Execution
# =================================================================

if agent_turn.requires_confirmation:
    print("✅ USER ACTION: Confirming plan...\n")

    # Execute the pending actions using the Tool Execution Agent
    # This mimics the "Act" step
    results = tool_agent.execute_actions(agent_turn.pending_actions)

    print("--- 🛠️ Tool Execution Results ---")
    for res in results:
        # Check for success/failure in the tool output
        status = res.get("status", "unknown")
        details = res.get("details", str(res))
        print(f"[{status.upper()}] {details}")

else:
    print("ℹ️ No actions required confirmation.")